In [1]:
from pathlib import Path
import shutil

import amici
import amici.petab_import
import amici.petab_objective
from more_itertools import one
import numpy as np
import petab                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             


import petab_timecourse
#from petab_timecourse import (
#    import_directory_of_componentwise_files,
#)

#model_name = 'Cuadros_HealthPlace2020'

#simulation_duration = 205  # simulated time
#output_density = 10  # number of output timepoints per simulated time unit

#petab_path = Path(model_name)

#componentwise_timecourses_path = petab_path / 'regimens'

petab_problem = petab_timecourse.Problem.from_yaml("Cuadros_HealthPlace2020/petab_training_SBML_Cuadros_HealthPlace2020.yaml")

FileNotFoundError: [Errno 2] No such file or directory: 'timecourse.tsv'

In [2]:
petab_yaml_location = str(one(petab_path.glob('*yaml')))
petab_problem = petab.Problem.from_yaml(str(one(petab_path.glob('*yaml'))))

import copy
original_petab_problem = copy.deepcopy(petab_problem)

In [3]:
timecourse_df, condition_df = import_directory_of_componentwise_files(
    componentwise_timecourses_path,
)
petab_problem = petab.Problem.from_yaml(petab_yaml_location)
petab_problem.condition_df = condition_df
petab_problem.timecourse_df = timecourse_df
petab_problem.sbml_model = petab_timecourse.sbml.add_timecourse_as_events(
    petab_problem,
    timecourse_id=one(petab_problem.timecourse_df.index),
)
petab_problem.condition_df = original_petab_problem.condition_df

[{'epsilon': 1.0}, {'epsilon': 0.2}, {'epsilon': 1.0}]
                                                            timecourse
timecourseId                                                          
timecourse_regimens  1.0:{'epsilon': 1.0};22.0:{'epsilon': 0.2};75....


AttributeError: 'dict' object has no attribute 'condition_id'

In [ ]:
#amici_model = amici.petab_import.import_petab_problem(petab_problem, force_compile=True)
amici_model = amici.petab_import.import_petab_problem(petab_problem)

In [ ]:
sim = amici.petab_objective.simulate_petab(petab_problem, amici_model)

In [ ]:
from amici.plotting import plotObservableTrajectories
import matplotlib.pyplot as plt
observable_indices=[amici_model.getObservableIds().index('observable_epsilon')]
plotObservableTrajectories(sim['rdatas'][0], model=amici_model, observable_indices=observable_indices)
plt.show()

In [ ]:
from libsbml import formulaToL3String
print('trigger\t\tcomponent\tvalue')
for e in petab_problem.sbml_model.getListOfEvents():
    print(formulaToL3String(e.getTrigger().getMath()), end='\t')
    for a in e.getListOfEventAssignments():
        print(a.getVariable(), end='\t\t')
        print(formulaToL3String(a.getMath()))

In [ ]:
import pypesto.petab
# Import the problem and compile the model.
_importer = pypesto.petab.PetabImporter.from_yaml(petab_yaml_location)
#pypesto_problem = _importer.create_problem(force_compile=True)
pypesto_problem = _importer.create_problem(force_compile=True)

In [ ]:
from amici.plotting import plotObservableTrajectories
import matplotlib.pyplot as plt

# Simulate and plot the timecourse parameter.
observable_indices=[pypesto_problem.objective.amici_model.getObservableIds().index('observable_epsilon')]

t = np.linspace(0, simulation_duration, simulation_duration*output_density + 1)
pypesto_problem.objective.amici_model.setTimepoints(t)

par_dict = pypesto_problem.objective.par_arr_to_dct(petab_problem.x_nominal)
ignored_pars = set(par_dict.keys()).difference(pypesto_problem.objective.amici_model.getParameterIds())
pypesto_problem.objective.amici_model.setParameterById({
    k: v
    for k, v in par_dict.items()
    if k not in ignored_pars
})
rdata = amici.runAmiciSimulation(
    pypesto_problem.objective.amici_model,
    pypesto_problem.objective.amici_solver,
)
plotObservableTrajectories(
    rdata,
    model=pypesto_problem.objective.amici_model,
    observable_indices=observable_indices,
)
plt.show()

In [ ]:
import pypesto.engine
import pypesto.optimize

#optimizer = pypesto.optimize.ScipyOptimizer()
#engine = pypesto.engine.MultiProcessEngine()

optimizer = pypesto.optimize.FidesOptimizer()
engine = pypesto.engine.MultiProcessEngine()

pypesto_result = pypesto.optimize.minimize(
    problem=pypesto_problem,
    optimizer=optimizer,
    n_starts=16,
    engine=engine,
)

In [ ]:
import pypesto.visualize
import matplotlib.pyplot as plt
# Plot optimization information.
pypesto.visualize.waterfall(pypesto_result)
pypesto.visualize.parameters(pypesto_result)
plt.show()

In [ ]:
# Simulate and plot the MLE
'''
from amici.plotting import plotStateTrajectories, plotObservableTrajectories
import matplotlib.pyplot as plt
t = np.linspace(0, simulation_duration, simulation_duration*output_density + 1)
pypesto_problem.objective.amici_model.setTimepoints(t)
for start in pypesto_result.optimize_result.list:
    par_dict = pypesto_problem.objective.par_arr_to_dct(np.power(10, start['x']))
    ignored_pars = set(par_dict.keys()).difference(pypesto_problem.objective.amici_model.getParameterIds())
    pypesto_problem.objective.amici_model.setParameterById({
        k: v
        for k, v in par_dict.items()
        if k not in ignored_pars
    })
    #pypesto_problem.objective.amici_model.setParameters(
    #    np.power(10, start['x']),
    #)
    #solver = model.getSolver()
    rdata = amici.runAmiciSimulation(
        pypesto_problem.objective.amici_model,
        pypesto_problem.objective.amici_solver,
    )
     
    #plotStateTrajectories(rdata, model=pypesto_problem.objective.amici_model)
    plotObservableTrajectories(
        rdata,
        model=pypesto_problem.objective.amici_model,
        observable_indices=observable_indices,
    )
    #plt.plot(t, [get_expected_x(t_i) for t_i in t])
    plt.show()
'''

In [ ]:
#print(pypesto_problem.objective.amici_model.getObservableIds())
for start in pypesto_result.optimize_result.list[:1]:
    observable_indices=[pypesto_problem.objective.amici_model.getObservableIds().index('observable_Total_cumulative_cases')]
    par_dict = pypesto_problem.objective.par_arr_to_dct(np.power(10, start['x']))
    ignored_pars = set(par_dict.keys()).difference(pypesto_problem.objective.amici_model.getParameterIds())
    pypesto_problem.objective.amici_model.setParameterById({
        k: v
        for k, v in par_dict.items()
        if k not in ignored_pars
    })
    #pypesto_problem.objective.amici_model.setParameters(
    #    np.power(10, start['x']),
    #)
    #solver = model.getSolver()
    pypesto_problem.objective.amici_model.setTimepoints(np.linspace(0, 80, 801))
    rdata = amici.runAmiciSimulation(
        pypesto_problem.objective.amici_model,
        pypesto_problem.objective.amici_solver,
    )
     
    #plotStateTrajectories(rdata, model=pypesto_problem.objective.amici_model)
    fig, ax = plt.subplots(figsize=(20,10))
    plotObservableTrajectories(
        rdata,
        model=pypesto_problem.objective.amici_model,
        observable_indices=observable_indices,
        ax=ax,
    )
    #ax.set_yscale('log')
    #plt.plot(t, [get_expected_x(t_i) for t_i in t])
    plt.show()

In [ ]:
pypesto_problem.objective.check_grad_multi_eps(x=petab_problem.x_nominal_free)